## How to do it...

1. Load necessary libraries

In [2]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions

2. Set parameters

In [3]:
batch_size = 2
buffer_size = 1000

3. Download CIFAR-10 data and declare categories

In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

objects = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

4. Initialize data pipeline

In [5]:
dataset_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
dataset_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))

5. Preprocess 

In [7]:
def preprocess_cifar10(img, label):
    img = tf.cast(img, tf.float32)
    img = tf.image.resize(img, (75, 75))
    
    return tf.keras.applications.inception_v3.preprocess_input(img), label

dataset_train_processed = dataset_train.shuffle(buffer_size).batch(batch_size).map(preprocess_cifar10)
dataset_test_processed = dataset_test.batch(batch_size).map(preprocess_cifar10)

6. Create model

In [8]:
inception_model = InceptionV3(
    include_top=False,
    weights='imagenet',
    input_shape=(75,75,3)
)

87924736/87910968 [==============================] - 1s 0us/step


7. Build on top of model

In [9]:
x = inception_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(1024, activation='relu')(x)
x = keras.layers.Dense(128, activation='relu')(x)
output = keras.layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs=inception_model.input, outputs=output)

8. Freeze inception layer parameters

In [10]:
for inception_layer in inception_model.layers:
    inception_layer.trainable=False

9. Compile model

In [11]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

10. Fit model

In [13]:
model.fit(x=dataset_train_processed, validation_data=dataset_test_processed)

25000/25000 [==============================] - 289s 11ms/step - loss: 1.2281 - accuracy: 0.5762 - val_loss: 1.0986 - val_accuracy: 0.6166
